In [1]:
# from slimstampen.spacingmodel import SpacingModel, Fact, Response
from slimstampen.spacingmodel import UIFeatures, Fact, Response

In [2]:
m = UIFeatures()

In [3]:
# reading from a CSV
import pandas as pd
# df = pd.read_table("./vocabulary_marked.csv", names=["id","context_1", "context_2", "answer"], index_col=0, sep = ",", lineterminator='\n')
df = pd.read_csv("./vocabulary_marked.csv")
df.head(4)

,id,question,context_1,context_2,answer
0,1,gemiddeld,Deze opleiding heeft 'gemiddeld' 60 studenten ...,De temperatuur in Nederland is 'gemiddeld' 15 ...,average
1,2,vervelend,Ik vind het 'vervelend' als de koffie op is.,Doe eens niet zo 'vervelend'.,annoying
2,3,algemeen,Dat is niet 'algemeen' bekend.,Over het 'algemeen' ben ik hier heel goed in.,general
3,4,matig,De burger smaakte een beetje 'matig'.,Ik ben 'matig' enthousiast over deze activiteit.,mediocre


In [4]:
UIFeatures().ret_translated_word("average")

'gemiddeld'

In [5]:
UIFeatures().word_from_context("Ik vind het 'vervelend' als de koffie op is.")

'vervelend'

In [6]:
# adding all facts to spacing model
m = UIFeatures()
for row in df.itertuples():
    m.add_fact(Fact(fact_id = row.id,context_1= row.context_1, context_2 = row.context_2, answer=row.answer, chosen_context=""))
m.facts[:2]

[Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context=''),
 Fact(fact_id=2, context_1="Ik vind het 'vervelend' als de koffie op is.", context_2="Doe eens niet zo 'vervelend'.", answer='annoying', chosen_context='')]

In [7]:
next_fact, new = m.get_next_fact(current_time = 34000)
next_fact, new

(Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context=''),
 True)

In [8]:
resp = Response(fact = next_fact, start_time = 34029, rt = 2207, correct = True)

m.register_response(resp)

In [9]:
m.responses

[Response(fact=Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context=''), start_time=34029, rt=2207, correct=True)]

### Asking for more facts

When it's time to present the next trial, ask for another fact. With default parameter settings, the model chooses to immediately reinforce the fact that was just introduced, because it is likely to be forgotten otherwise. Note that this fact is now no longer marked as 'new'.

In [10]:
next_fact, new = m.get_next_fact(current_time = 38000)
next_fact, new

(Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context=''),
 False)

Behind the scenes, the `get_next_fact` method has checked the expected activation of all facts slightly in the future (by default the model looks ahead 15 seconds). Since the activation of the first fact is predicted to be lower than the threshold of -0.8, it is selected for repetition. Note that facts that have not yet been studied have an activation of $-\infty$. 

In [11]:
for f in m.facts:
    print(f, m.calculate_activation(38000 + m.LOOKAHEAD_TIME, f))

Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context='') -0.8828734492111224
Fact(fact_id=2, context_1="Ik vind het 'vervelend' als de koffie op is.", context_2="Doe eens niet zo 'vervelend'.", answer='annoying', chosen_context='') -inf
Fact(fact_id=3, context_1="Dat is niet 'algemeen' bekend.", context_2="Over het 'algemeen' ben ik hier heel goed in.", answer='general', chosen_context='') -inf
Fact(fact_id=4, context_1="De burger smaakte een beetje 'matig'.", context_2="Ik ben 'matig' enthousiast over deze activiteit.", answer='mediocre', chosen_context='') -inf
Fact(fact_id=5, context_1="Ze 'verwachten' dat het morgen gaat regenen.", context_2="Wij 'verwachten' dat jij het goed gaat doen.", answer='expect', chosen_context='') -inf
Fact(fact_id=6, context_1="Ben je nou alweer je sleutel 'vergeten'?", context_2="Sorry, we waren 'vergeten' de wiki aan te pa

Let's say this fact is again presented to the user and a correct response is made.

In [12]:
resp = Response(fact = next_fact, start_time = 38007, rt = 1890, correct = True)

m.register_response(resp)

The additional successful repetition means that the activation of this fact will now still be high enough in 15 seconds. 

In [13]:
for f in m.facts:
    print(f, m.calculate_activation(42000 + m.LOOKAHEAD_TIME, f))

Fact(fact_id=1, context_1="Deze opleiding heeft 'gemiddeld' 60 studenten per jaar.", context_2="De temperatuur in Nederland is 'gemiddeld' 15 graden.", answer='average', chosen_context='') -0.4389281810299998
Fact(fact_id=2, context_1="Ik vind het 'vervelend' als de koffie op is.", context_2="Doe eens niet zo 'vervelend'.", answer='annoying', chosen_context='') -inf
Fact(fact_id=3, context_1="Dat is niet 'algemeen' bekend.", context_2="Over het 'algemeen' ben ik hier heel goed in.", answer='general', chosen_context='') -inf
Fact(fact_id=4, context_1="De burger smaakte een beetje 'matig'.", context_2="Ik ben 'matig' enthousiast over deze activiteit.", answer='mediocre', chosen_context='') -inf
Fact(fact_id=5, context_1="Ze 'verwachten' dat het morgen gaat regenen.", context_2="Wij 'verwachten' dat jij het goed gaat doen.", answer='expect', chosen_context='') -inf
Fact(fact_id=6, context_1="Ben je nou alweer je sleutel 'vergeten'?", context_2="Sorry, we waren 'vergeten' de wiki aan te pa

Sure enough, when we ask for a new fact, we get a different one:

In [14]:
next_fact, new = m.get_next_fact(current_time = 42000)
next_fact, new

(Fact(fact_id=2, context_1="Ik vind het 'vervelend' als de koffie op is.", context_2="Doe eens niet zo 'vervelend'.", answer='annoying', chosen_context=''),
 True)

### Estimated *rate of forgetting* ($\alpha$)

A fact's rate of forgetting ($\alpha$) always starts at 0.3. Once at least 3 responses have been recorded, this value is adjusted up (if the fact is difficult to remember) or down (if it's easy to remember).

The `get_rate_of_forgetting` method returns the estimated rate of forgetting for a fact at a specified time, given the responses that were made before that time.

We can confirm that the rate of forgetting estimate for the first fact is indeed 0.3 before any responses have been logged:  

In [15]:
print('At t=0: {}'.format(m.get_rate_of_forgetting(0, fact=next_fact)))

At t=0: 0.3


If we add some more responses for `fact1` we can see how adjustments in the estimated rate of forgetting happen (notice that adjustment only starts after response #3). In this case, the observed response times are lower than expected, so the rate of forgetting estimate is adjusted downwards to about 0.20.

In [16]:
print('After 2 responses: {}'.format(m.get_rate_of_forgetting(50000, next_fact)))

resp = Response(fact = next_fact, start_time = 50000, rt = 1200, correct = True)
m.register_response(resp)

print('After 3 responses: {}'.format(m.get_rate_of_forgetting(60000, next_fact)))

resp = Response(fact = next_fact, start_time = 60000, rt = 1100, correct = False)
m.register_response(resp)

print('After 4 responses: {}'.format(m.get_rate_of_forgetting(70000, next_fact)))

resp = Response(fact = next_fact, start_time = 70000, rt = 1000, correct = True)
m.register_response(resp)

print('After 5 responses: {}'.format(m.get_rate_of_forgetting(80000, next_fact)))


After 2 responses: 0.3
After 3 responses: 0.3
After 4 responses: 0.3
After 5 responses: 0.29960937499999996


### Exporting the response data

The method `export_data` provides a simple way of saving the response data and the model estimates. It also returns a copy of the data in case you want to do more with it.

The column `alpha` contains the estimated rate of forgetting *after* the trial.

In [17]:
m.export_data("data.csv")

,start_time,rt,correct,fact_id,context_1,context_2,answer,chosen_context,alpha,reading_time
trial,,,,,,,,,,
1,34029,2207,True,1,Deze opleiding heeft 'gemiddeld' 60 studenten ...,De temperatuur in Nederland is 'gemiddeld' 15 ...,average,,0.300000,914.6
2,38007,1890,True,1,Deze opleiding heeft 'gemiddeld' 60 studenten ...,De temperatuur in Nederland is 'gemiddeld' 15 ...,average,,0.300000,914.6
3,50000,1200,True,2,Ik vind het 'vervelend' als de koffie op is.,Doe eens niet zo 'vervelend'.,annoying,,0.300000,700.1
4,60000,1100,False,2,Ik vind het 'vervelend' als de koffie op is.,Doe eens niet zo 'vervelend'.,annoying,,0.300000,700.1
5,70000,1000,True,2,Ik vind het 'vervelend' als de koffie op is.,Doe eens niet zo 'vervelend'.,annoying,,0.299609,700.1


In [19]:
print(m.achievement())


        

        Trial Index : RangeIndex(start=1, stop=6, step=1, name='trial')
        Total Correct : 4
        Total Wrong : 1
        Mean Correct : 0.8
        Mean Wrong : 0.2
        
